In [ ]:
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Mecab
import re
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Step 1: 데이터 로드
file_path = "C:/Users/cho03/Desktop/대학/4학년/2학기/캡스톤디자인-여행사이트/data/lodging_reviews.csv"
data = pd.read_csv(file_path)
data = data.replace("\n", " ", regex=True)  # 개행 문자를 공백으로 대체

# Step 2: 'review_text' 또는 'rating' 컬럼에 NaN이 있는 행 삭제
data = data.dropna(subset=["review_text", "rating"])

# 리뷰 텍스트의 길이를 계산하여 'length' 컬럼에 저장
data["length"] = data["review_text"].str.len()

# Step 3: 특수 문자 제거 (한글, 영어, 숫자만 유지)
data["review_text"] = data["review_text"].apply(lambda x: re.sub("[^0-9a-zA-Zㄱ-ㅣ가-힣 ]", "", str(x)))


# Step 4: 의미없는 자음, 모음만 쓴 것 제거
def cleanText(readData):
    text = re.sub("[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]", "", readData)
    return text


data["review_text"] = data["review_text"].apply(cleanText)

# 리뷰의 길이가 10 미만인 데이터 제거
data = data[data["length"] >= 10]

# Step 5: 각 숙소(lodging_id)별 전체 리뷰를 기준으로 평균 평점과 리뷰 개수 계산
lodging_summary = (
    data.groupby("lodging_id")
    .agg(total_review_count=("rating", "count"), average_rating=("rating", "mean"))
    .reset_index()
)

# 조건 1: 전체 리뷰의 평균 평점이 4 이상인 숙소
qualified_lodging_ids = lodging_summary[lodging_summary["average_rating"] >= 4]["lodging_id"]

# Step 6: 조건을 만족하는 숙소 중 평점이 4 이상인 리뷰만 필터링
good_review = data[(data["lodging_id"].isin(qualified_lodging_ids)) & (data["rating"] >= 4)]

# Step 7: 평점이 4 이상인 리뷰 개수가 5개 이상인 숙소만 선택
final_lodging_summary = good_review.groupby("lodging_id").agg(good_review_count=("rating", "count")).reset_index()

# 평점 4 이상 리뷰가 5개 이상인 숙소 ID 추출
final_lodging_ids = final_lodging_summary[final_lodging_summary["good_review_count"] >= 5]["lodging_id"]

# Step 8: 최종 필터링 - 전체 조건을 만족하는 숙소의 평점 4 이상인 리뷰만 추출
filtered_data = good_review[good_review["lodging_id"].isin(final_lodging_ids)]

In [ ]:
# 불용어 처리
mecab = Mecab(dicpath="..\mecab\mecab-ko-dic")
f = open("stopwords-ko.txt", "r", encoding="UTF-8")
st = f.readlines()
f.close()

# 줄바꿈 문자 제거
stw = []
for word in tqdm(st, desc="불용어 목록 정리"):
    cleaned_word = word.strip()
    stw.append(cleaned_word)

# 사용자가 추가한 불용어 리스트 병합
user_stopwords = []
stw.extend(user_stopwords)
stopwords = list(set(stw))
for i in tqdm(range(len(filtered_data)), desc="품사 추출 중"):
    review_text = filtered_data["review_text"].iloc[i]
    tokens = []
    for word, pos in mecab.pos(review_text):
        if pos in ["Noun", "Adjective", "Verb"]:
            print(tokens)
            tokens.append(word)

In [ ]:
# 품사 추출 및 불용어 제거
tokenized_text = []
for i in tqdm(range(len(filtered_data)), desc="품사 추출 중"):
    review_text = filtered_data["review_text"].iloc[i]
    tokens = []
    for word, pos in mecab.pos(review_text):
        if pos in ["NNG", "NNP", "NNB", "VA", "XR", "VV", "VX"]:
            tokens.append(word)
    tokenized_text.append(tokens)


def remove_stopwords(tokenized_reviews, stopwords):
    cleaned_reviews = []  # 빈 리스트 초기화

    # 모든 리뷰의 토큰 리스트를 반복
    for tokens in tokenized_reviews:
        cleaned_tokens = []  # 불용어가 제거된 현재 리뷰의 토큰 리스트 초기화

        # 현재 리뷰의 각 단어를 순회하며 불용어 검사
        for word in tokens:
            # 단어가 불용어 목록에 없으면 cleaned_tokens 리스트에 추가
            if word not in stopwords:
                cleaned_tokens.append(word)

        # 불용어가 제거된 토큰 리스트를 전체 리뷰 리스트에 추가
        cleaned_reviews.append(cleaned_tokens)

    return cleaned_reviews  # 불용어가 제거된 리뷰 리스트 반환


filtered_data = filtered_data.copy()  # 명시적으로 복사본 생성
filtered_data["review_after"] = remove_stopwords(tokenized_text, stopwords)

In [ ]:
filtered_data

In [ ]:
# KoBERT 모델 로드 및 태그 예시 문장 정의
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

tag_examples = {
    "가성비": [
        "가격 대비 만족스러운 숙소입니다.",
        "비용에 비해 기대 이상이었어요.",
        "저렴한 가격에 만족스러웠습니다.",
        "합리적인 가격으로 숙박하기 좋았습니다.",
        "가격이 저렴하면서 필요한 시설이 잘 갖춰져 있어요.",
        "가성비가 좋아요.",
        "가격 대비 품질이 좋습니다.",
        "가격대비 만족합니다.",
        "비용에 비해 적절한 선택이었어요.",
        "가격 부담 없이 즐길 수 있는 숙소였습니다.",
    ],
    "청결": [
        "객실이 매우 깨끗하게 유지되고 있었습니다.",
        "전체적으로 깔끔하게 관리되고 있어요.",
        "청결 상태가 좋아서 안심이 됩니다.",
        "방과 욕실 모두 깨끗했습니다.",
        "숙소가 위생적으로 관리되고 있어요.",
        "깔끔하고 청결한 공간이었습니다.",
        "정돈이 잘 되어 있어요.",
        "청결도가 좋아 만족했습니다.",
        "깨끗하고 위생적입니다.",
        "숙소가 깔끔하게 관리되고 있습니다.",
        "위생 관리가 철저한 느낌이었습니다.",
        "쾌적한 환경을 유지하는 숙소였습니다.",
    ],
    "직원 만족": [
        "직원들이 정말 친절하게 응대해주셨어요.",
        "서비스가 매우 세심하고 좋았습니다.",
        "프론트 직원이 특히 친절했어요.",
        "요청 사항에 빠르게 대응해 주셨습니다.",
        "직원들의 서비스 마인드가 훌륭합니다.",
        "친절하고 상냥한 직원들이 있습니다.",
        "직원들이 매우 협조적입니다.",
        "친절한 서비스 덕분에 편안하게 머물렀습니다.",
        "직원 서비스가 인상적이었어요.",
        "서비스 수준이 높습니다.",
    ],
    "위치": [
        "위치가 좋아 이동하기 편리했습니다.",
        "관광지와 가까워서 좋았어요.",
        "대중교통과 접근성이 좋습니다.",
        "중심지에 위치해 주변 시설 이용이 편리했습니다.",
        "조용한 곳에 위치해 휴식하기 좋았어요.",
        "교통이 편리해요.",
        "중심지와 가까워요.",
        "위치가 좋아서 접근성이 좋습니다.",
        "이동하기에 매우 편리합니다.",
        "주변 시설 접근성이 훌륭해요.",
    ],
    "가족 여행": [
        "아이들과 머물기에 좋은 숙소입니다.",
        "가족 단위로 이용하기에 편리해요.",
        "가족 여행에 최적화된 시설입니다.",
        "가족이 함께 즐길 수 있는 공간이 있어 좋았습니다.",
        "아이들이 즐길 수 있는 편의 시설이 잘 갖춰져 있어요.",
        "가족 여행에 좋습니다.",
        "가족끼리 오기 좋았어요.",
        "아이들과 함께 지내기 좋아요.",
        "아이들과 함께 이용하기에 안성맞춤입니다.",
    ],
    "연인": [
        "로맨틱한 분위기로 커플 여행에 좋았습니다.",
        "연인과 함께하기에 이상적인 숙소예요.",
        "특별한 날에 방문하기 좋은 곳입니다.",
        "아늑한 분위기가 좋아 연인과 추천합니다.",
        "연인과 특별한 시간을 보내기 좋았습니다.",
        "커플에게 딱 맞는 숙소예요.",
        "로맨틱한 분위기가 좋았어요.",
        "연인과 함께할 분위기였습니다.",
        "둘만의 특별한 시간을 보내기 좋은 곳입니다.",
    ],
    "풍경": [
        "자연 경관이 멋진 숙소입니다.",
        "탁 트인 전망이 인상적이에요.",
        "숙소 주변의 풍경이 아름다웠습니다.",
        "뷰가 좋아 마음이 편안해졌어요.",
        "산과 호수의 멋진 뷰를 즐길 수 있어 좋았어요.",
        "풍경이 아름다워요.",
        "경치가 좋아서 힐링됩니다.",
        "멋진 전망을 감상할 수 있어요.",
        "뷰가 정말 인상 깊었습니다.",
    ],
}

In [ ]:
# 태그 예시 문장 임베딩 생성 (풀어서 작성) with tqdm
tag_embeddings = {}  # 빈 딕셔너리 초기화

for tag, sentences in tqdm(tag_examples.items(), desc="예시 문장 임베딩 중"):
    embeddings = model.encode(sentences)  # 현재 태그의 문장 리스트를 임베딩 생성
    tag_embeddings[tag] = embeddings  # 생성된 임베딩을 태그 키에 저장

# 리뷰 문장 임베딩 생성
review_embeddings = []
for tokens in tqdm(filtered_data["review_after"], desc="리뷰 문장 임베딩 중"):
    review_text = " ".join(tokens)  # 토큰을 하나의 문자열로 결합
    embedding = model.encode(review_text)  # 결합된 문자열을 임베딩으로 변환
    review_embeddings.append(embedding)

In [ ]:
# 유사도 임계값 설정
threshold = 0.5
assigned_tags_per_review = []
for review_embedding in tqdm(review_embeddings, desc="리뷰별 태그 할당 중"):
    tag_scores = {}

    # 각 태그의 임베딩과 리뷰 임베딩 간 유사도 계산
    for tag, embeddings in tag_embeddings.items():
        similarity = cosine_similarity([review_embedding], embeddings).mean()  # 유사도 평균 계산
        tag_scores[tag] = similarity  # 태그별 유사도 저장

    # 유사도 점수가 임계값보다 큰 태그를 리뷰에 할당
    review_tags = []
    for tag, score in tag_scores.items():
        if score > threshold:
            review_tags.append(tag)

    assigned_tags_per_review.append(review_tags)

In [ ]:
# 할당된 태그를 데이터프레임에 추가
filtered_data["assigned_tags"] = assigned_tags_per_review

# 리뷰별 태그 빈도 계산 (풀어서 작성)
review_tag_counts = []
for tags in tqdm(filtered_data["assigned_tags"], desc="리뷰별 태그 및 빈도 계산"):
    tag_counts = Counter(tags)  # 각 리뷰의 태그 빈도 계산
    review_tag_counts.append(
        {
            "리뷰의_가성비": tag_counts.get("가성비", 0),
            "리뷰의_청결도": tag_counts.get("청결", 0),
            "리뷰의_직원_만족": tag_counts.get("직원 만족", 0),
            "리뷰의_위치": tag_counts.get("위치", 0),
            "리뷰의_가족_여행": tag_counts.get("가족 여행", 0),
            "리뷰의_연인": tag_counts.get("연인", 0),
            "리뷰의_풍경": tag_counts.get("풍경", 0),
        }
    )

# 리뷰별 태그 빈도 리스트를 데이터프레임으로 변환
review_tag_counts_df = pd.DataFrame(review_tag_counts)

# 리뷰 데이터에 리뷰별 태그 빈도 추가
final_review_data = pd.concat([filtered_data.reset_index(drop=True), review_tag_counts_df], axis=1)

In [ ]:
# 숙소별 태그 빈도 계산 및 상위 2개 태그 선정
final_tags_per_lodging = []
lodging_ids = final_review_data["lodging_id"].unique()

for lodging_id in tqdm(lodging_ids, desc="숙소별 태그 및 빈도 계산"):
    keywords_all_reviews = []
    for tags in final_review_data[final_review_data["lodging_id"] == lodging_id]["assigned_tags"]:
        keywords_all_reviews.extend(tags)

    # 태그 빈도 계산
    tag_counts = Counter(keywords_all_reviews)
    top_2_tags = []

    # 상위 2개의 태그 중 빈도가 3 이상인 경우만 추가
    for tag, count in tag_counts.most_common():
        if count >= 3:  # 빈도 조건을 충족하는지 확인
            top_2_tags.append(tag)
        if len(top_2_tags) == 2:  # 상위 2개 태그가 채워지면 종료
            break

    # 각 숙소의 태그 빈도 및 상위 태그 추가
    final_tags_per_lodging.append(
        {
            "lodging_id": lodging_id,
            "숙소의_가성비": tag_counts.get("가성비", 0),
            "숙소의_청결도": tag_counts.get("청결", 0),
            "숙소의_직원_만족": tag_counts.get("직원 만족", 0),
            "숙소의_위치": tag_counts.get("위치", 0),
            "숙소의_가족_여행": tag_counts.get("가족 여행", 0),
            "숙소의_연인": tag_counts.get("연인", 0),
            "숙소의_풍경": tag_counts.get("풍경", 0),
            "숙소의_top_2_tags": top_2_tags,
        }
    )

In [ ]:
# 최종 숙소별 태그 빈도를 데이터프레임으로 변환하고 원래 데이터와 병합
lodging_tags_specified_df = pd.DataFrame(final_tags_per_lodging)
final_data = pd.merge(final_review_data, lodging_tags_specified_df, on="lodging_id", how="left")
final_data

In [ ]:
final_data.to_csv("review_tags(mecab).csv", encoding="utf-8-sig", index=False)